In [1]:
!apt install libavdevice-dev libavfilter-dev libopus-dev libvpx-dev pkg-config
#!pip install aiohttp aiortc opencv-python websockets

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libavdevice-dev
E: Unable to locate package libavfilter-dev


In [ ]:
#!apt install libavdevice-dev libavfilter-dev libopus-dev libvpx-dev pkg-config
!pip install aiohttp aiortc opencv-python websockets

In [ ]:
import argparse
import asyncio
import json
import logging
import os
import random

import aiohttp
import cv2
import websockets
from av import VideoFrame

from aiortc import (RTCIceCandidate, RTCPeerConnection, RTCSessionDescription,
                    VideoStreamTrack)
from aiortc.contrib.media import MediaBlackhole, MediaPlayer, MediaRecorder
from aiortc.contrib.signaling import object_from_string, object_to_string

ROOT = os.path.dirname('.')
print(ROOT)
PHOTO_PATH = os.path.join(ROOT, 'photo.jpg')


class ApprtcSignaling:
    def __init__(self, room):
        self._http = None
        self._origin = 'https://appr.tc'
        self._room = room
        self._websocket = None

    async def connect(self):
        join_url = self._origin + '/join/' + self._room

        # fetch room parameters
        self._http = aiohttp.ClientSession()
        async with self._http.post(join_url) as response:
            # we cannot use response.json() due to:
            # https://github.com/webrtc/apprtc/issues/562
            data = json.loads(await response.text())
        assert data['result'] == 'SUCCESS'
        params = data['params']

        self.__is_initiator = params['is_initiator'] == 'true'
        self.__messages = params['messages']
        self.__post_url = self._origin + '/message/' + self._room + '/' + params['client_id']

        # connect to websocket
        self._websocket = await websockets.connect(params['wss_url'], extra_headers={
            'Origin': self._origin
        })
        await self._websocket.send(json.dumps({
            'clientid': params['client_id'],
            'cmd': 'register',
            'roomid': params['room_id'],
        }))

        return params

    async def close(self):
        if self._websocket:
            await self.send(None)
            self._websocket.close()
        if self._http:
            await self._http.close()

    async def receive(self):
        if self.__messages:
            message = self.__messages.pop(0)
        else:
            message = await self._websocket.recv()
            message = json.loads(message)['msg']
        print('<', message)
        return object_from_string(message)

    async def send(self, obj):
        message = object_to_string(obj)
        print('>', message)
        if self.__is_initiator:
            await self._http.post(self.__post_url, data=message)
        else:
            await self._websocket.send(json.dumps({
                'cmd': 'send',
                'msg': message,
            }))

class VideoTransformTrack(VideoStreamTrack):
    def __init__(self, track, transform):
        super().__init__()  # don't forget this!
        self.counter = 0
        self.track = track
        self.transform = transform

    async def recv(self):
        frame = await self.track.recv()
        self.counter += 1

        if self.transform == 'cartoon':
            img = frame.to_ndarray(format='bgr24')

            # prepare color
            img_color = cv2.pyrDown(cv2.pyrDown(img))
            for _ in range(6):
                img_color = cv2.bilateralFilter(img_color, 9, 9, 7)
            img_color = cv2.pyrUp(cv2.pyrUp(img_color))

            # prepare edges
            img_edges = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            img_edges = cv2.adaptiveThreshold(
                cv2.medianBlur(img_edges, 7), 255,
                cv2.ADAPTIVE_THRESH_MEAN_C,
                cv2.THRESH_BINARY, 9, 2)
            img_edges = cv2.cvtColor(img_edges, cv2.COLOR_GRAY2RGB)

            # combine color and edges
            img = cv2.bitwise_and(img_color, img_edges)

            # rebuild a VideoFrame, preserving timing information
            new_frame = VideoFrame.from_ndarray(img, format='bgr24')
            new_frame.pts = frame.pts
            new_frame.time_base = frame.time_base
            return new_frame
        elif self.transform == 'edges':
            # perform edge detection
            img = frame.to_ndarray(format='bgr24')
            img = cv2.cvtColor(cv2.Canny(img, 100, 200), cv2.COLOR_GRAY2BGR)

            # rebuild a VideoFrame, preserving timing information
            new_frame = VideoFrame.from_ndarray(img, format='bgr24')
            new_frame.pts = frame.pts
            new_frame.time_base = frame.time_base
            return new_frame
        elif self.transform == 'rotate':
            # rotate image
            img = frame.to_ndarray(format='bgr24')
            rows, cols, _ = img.shape
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), frame.time * 45, 1)
            img = cv2.warpAffine(img, M, (cols, rows))

            # rebuild a VideoFrame, preserving timing information
            new_frame = VideoFrame.from_ndarray(img, format='bgr24')
            new_frame.pts = frame.pts
            new_frame.time_base = frame.time_base
            return new_frame
        else:
            return frame

class VideoImageTrack(VideoStreamTrack):
    """
    A video stream track that returns a rotating image.
    """
    def __init__(self):
        super().__init__()  # don't forget this!
        self.img = cv2.imread(PHOTO_PATH, cv2.IMREAD_COLOR)

    async def recv(self):
        pts, time_base = await self.next_timestamp()

        # rotate image
        rows, cols, _ = self.img.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), int(pts * time_base * 45), 1)
        img = cv2.warpAffine(self.img, M, (cols, rows))

        # create video frame
        frame = VideoFrame.from_ndarray(img, format='bgr24')
        frame.pts = pts
        frame.time_base = time_base

        return frame


async def run(pc, player, recorder, signaling):
    def add_tracks():
        
      #if player and player.audio:
      #    pc.addTrack(player.audio)

      if player and player.video:
          local_video = player.video
          local_video = VideoTransformTrack(player.video, transform='edges')
          #local_video = VideoTransformTrack(player.video, transform='cartoon')
          pc.addTrack(local_video)
      else:
          pc.addTransceiver('video','sendrecv') #this is the trick to echo webcame back
          #pc.addTrack(local_video) #VideoImageTrack())

#     @pc.on('track')
#     def on_track(track):
#         print('Track %s received' % track.kind)
#         recorder.addTrack(track)

    @pc.on('track')
    def on_track(track):
        #log_info('Track %s received', track.kind)
        print('Track %s received' % track.kind)
        if track.kind == 'audio':
            #pc.addTrack(player.audio)
            recorder.addTrack(track)
        elif track.kind == 'video':
            remote_video = track
            remote_video = VideoTransformTrack(track, transform='edges')
            #remote_video = VideoTransformTrack(track, transform='rotate')
            pc.addTrack(remote_video)
            recorder.addTrack(remote_video)

        @track.on('ended')
        async def on_ended():
            #log_info('Track %s ended', track.kind)
            await recorder.stop()


    # connect to websocket and join
    params = await signaling.connect()

    if params['is_initiator'] == 'true':
        # send offer
        add_tracks()
        await pc.setLocalDescription(await pc.createOffer())
        await signaling.send(pc.localDescription)
        print('Please point a browser at %s' % params['room_link'])

    # consume signaling
    while True:
        obj = await signaling.receive()

        if isinstance(obj, RTCSessionDescription):
            await pc.setRemoteDescription(obj)
            await recorder.start()

            if obj.type == 'offer':
                # send answer
                add_tracks()
                
#                 print('Add processed_video track')
#                 processed_video = VideoTransformTrack(pc.getSenders()[0].track, transform='edges')
#                 pc.addTrack(processed_video) #VideoImageTrack())

                await pc.setLocalDescription(await pc.createAnswer())
                await signaling.send(pc.localDescription)
        elif isinstance(obj, RTCIceCandidate):
            pc.addIceCandidate(obj)
        else:
            print('Exiting')
            break


if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='AppRTC')
#     parser.add_argument('room', nargs='?')
#     parser.add_argument('--play-from', help='Read the media from a file and sent it.'),
#     parser.add_argument('--record-to', help='Write received media to a file.'),
#     parser.add_argument('--verbose', '-v', action='count')
#     args = parser.parse_args()
  
    args_room = None #"test1234test1"
    args_play_from = None
    args_play_from = 'video6.mp4'
    args_play_from = 'colorbars.mp4'
    #args_play_from = 'video14-best.mp4'
    args_play_from = None
    args_record_to = None
    #args_record_to = '/content/video7.mp4'
    
    if not args_room:
        args_room = ''.join([random.choice('0123456789') for x in range(10)])

#     if args_verbose:
#         logging.basicConfig(level=logging.DEBUG)

     # create signaling and peer connection
    signaling = ApprtcSignaling(args_room)
    pc = RTCPeerConnection()

    # create media source
    if args_play_from:
        player = MediaPlayer(args_play_from)
    else:
        player = None

    # create media sink
    if args_record_to:
        recorder = MediaRecorder(args_record_to)
    else:
        recorder = MediaBlackhole()

    # run event loop
    loop = asyncio.get_event_loop()
    try:
        loop.run_until_complete(run(
            pc=pc,
            player=player,
            recorder=recorder,
            signaling=signaling))
    except KeyboardInterrupt:
        pass
    finally:
        # cleanup
        loop.run_until_complete(recorder.stop())
        loop.run_until_complete(signaling.close())
        loop.run_until_complete(pc.close())


In [7]:
import pylab
import imageio
import cv2, pafy

url = "https://www.youtube.com/watch?v=uuQlMCMT71I"
#url = "https://www.youtube.com/watch?v=OHKAkhdQ6B4"
videoPafy = pafy.new(url)
best = videoPafy.getbest(preftype="webm")
best = videoPafy.getbest(preftype="mp4")

#imageio.setUseCache(false)

#on server run this command:
#ffplay -f mjpeg tcp://0.0.0.0:45654?listen
#clientsocket=socket.socket(socket.AF_INET,socket.SOCK_STREAM)
#clientsocket.connect((ip,45654)) #the target ip is where the ffplay is listening

#filename = 'http://techslides.com/demos/sample-videos/small.mp4'
filename = best.url
#vid = imageio.get_reader(filename,  'ffmpeg')
nums = range(1,1000)

cap = cv2.VideoCapture(filename)

#for image in vid:
#for i, image in enumerate(vid):
for num in nums:
    flag, image = cap.retrieve()   
    #image = vid.reader.get_next_data()
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    data = cv2.imencode('.jpg', gray)[1].tobytes()
    #clientsocket.send(data)
    fig = pylab.figure()
    fig.suptitle('image #{}'.format(num), fontsize=20)
    pylab.imshow(gray)
    pylab.show()

error: OpenCV(4.1.0-pre) /data/data/com.termux/files/home/build/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
